## Ingestion to PostgreSQL

### Set environment variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

pg_host  = os.getenv("POSTGRESQL_HOST")
if pg_host is None or pg_host == "":
    print("POSTGRESQL_HOST environment variable not set.")
    exit()

pg_user  = os.getenv("POSTGRESQL_USERNAME")
if pg_user is None or pg_user == "":
    print("POSTGRESQL_USERNAME environment variable not set.")
    exit()

pg_password  = os.getenv("POSTGRESQL_PASSWORD")
if pg_password is None or pg_password == "":
    print("POSTGRESQL_PASSWORD environment variable not set.")
    exit()

db_name  = os.getenv("POSTGRESQL_DATABASE")
if db_name is None or db_name == "":
    print("POSTGRESQL_DATABASE environment variable not set.")
    exit()

text_table_name = 'text_sample'
doc_table_name = 'doc_sample'
image_table_name = 'image_sample'

postgresql_params = {
    "host": pg_host,
    "port": "5432", 
    "dbname": db_name,
    "user": pg_user,
    "password": pg_password
}

#### Create table

In [ ]:
import psycopg2 

connection = psycopg2.connect(**postgresql_params)
print("Connection established.")



## Create text_sample table
table_schema_data = """
    id smallint PRIMARY KEY,
    title text,
    content text,
    category text,
    title_vector VECTOR(1536),
    content_vector VECTOR(1536)
 """

cursor = connection.cursor()
cursor.execute("CREATE EXTENSION IF NOT EXISTS vector"); 

connection.commit()
print("Adding extension")


cursor.execute(f"DROP TABLE IF  EXISTS {text_table_name} ")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {text_table_name} ({table_schema_data});")
connection.commit()
print("Table text_sample created.")
cursor.close()

## Create doc_sample table
table_schema_data = """
    id smallint PRIMARY KEY,
    chunk_content text,
    chunk_content_vector VECTOR(1536)
 """

cursor = connection.cursor()
cursor.execute(f"DROP TABLE IF  EXISTS {doc_table_name} ")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {doc_table_name} ({table_schema_data});")
connection.commit()
print("Table text_sample created.")
cursor.close()

## Create doc_sample table
table_schema_data = """
    id smallint PRIMARY KEY,
    image text,
    image_vector VECTOR(1024)
 """

cursor = connection.cursor()
cursor.execute(f"DROP TABLE IF  EXISTS {image_table_name} ")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {image_table_name} ({table_schema_data});")
connection.commit()
print("Table text_sample created.")
cursor.close()

connection.close()
print("close connection.")

#### Ingest text sample with embeddings

In [ ]:
import pandas as pd

text_df = pd.read_json('../data/text/product_docs_embeddings.json')

connection = psycopg2.connect(**postgresql_params)
print("Connection established.")

insert_sql = f"INSERT INTO {text_table_name}(id, title, content, category, title_vector, content_vector) VALUES(%s, %s, %s, %s, %s, %s)"

records = text_df.values.tolist()
cursor = connection.cursor()
cursor.executemany(insert_sql, records)

connection.commit()
print("Text sample ingested.")


cursor.close()
connection.close()

#### Ingest doc sample with embeddings

In [ ]:
import pandas as pd

doc_df = pd.read_json('../data/docs/employee_handbook_embeddings.json')

connection = psycopg2.connect(**postgresql_params)
print("Connection established.")

insert_sql = f"INSERT INTO {doc_table_name}(id, chunk_content, chunk_content_vector) VALUES(%s, %s, %s)"

records = doc_df.values.tolist()
cursor = connection.cursor()
cursor.executemany(insert_sql, records)

connection.commit()
print("Doc sample ingested.")

cursor.close()
connection.close()

#### Ingest image sample with embeddings

In [ ]:
import pandas as pd

image_df = pd.read_json('../data/images/images_embeddings.json')

connection = psycopg2.connect(**postgresql_params)
print("Connection established.")

insert_sql = f"INSERT INTO {image_table_name}(id, image, image_vector) VALUES(%s, %s, %s)"

records = image_df.values.tolist()
cursor = connection.cursor()
cursor.executemany(insert_sql, records)

connection.commit()
print("Image sample ingested.")

cursor.close()
connection.close()

### Create Index

In [ ]:
# Import the psycopg2 library if you're using PostgreSQL
import psycopg2
import math

connection = psycopg2.connect(**postgresql_params)
cursor = connection.cursor()
print("Connection established.")


##Checking if the table has more than 1000 of rows so it can be divided in lists while using ivfflat
##https://github.com/pgvector/pgvector#ivfflat

# Count the number of rows 
cursor.execute(f'SELECT count(*) FROM {text_table_name}')
rows = cursor.fetchone()[0]


if rows >= 1000:
    # Determine the number of lists based on the number of rows
    if rows <= 1000000:
        lists = rows / 1000
    else:
        lists = math.sqrt(rows)

    index_schema_text = f"""
        CREATE INDEX IF NOT EXISTS ix_title_vector_cosine
        ON {text_table_name}
        USING ivfflat (title_vector vector_cosine_ops)
        WITH (lists = {int(lists)});

        CREATE INDEX IF NOT EXISTS ix_content_vector_cosine
        ON {text_table_name}
        USING ivfflat (content_vector vector_cosine_ops)
        WITH (lists = {int(lists)});

    """
else:

    # For the text_sample table
    index_schema_text = f"""
        CREATE INDEX IF NOT EXISTS ix_title_vector_cosine ON {text_table_name} USING ivfflat  (title_vector vector_cosine_ops);
        CREATE INDEX IF NOT EXISTS ix_content_vector_cosine ON {text_table_name} USING ivfflat  (content_vector vector_cosine_ops);
    """


# Count the number of rows 
cursor.execute(f'SELECT count(*) FROM {doc_table_name}')
rows = cursor.fetchone()[0]

if rows >= 1000:
    # Determine the number of lists based on the number of rows
    if rows <= 1000000:
        lists = rows / 1000
    else:
        lists = math.sqrt(rows)

    index_schema_text = f"""
        CREATE INDEX IF NOT EXISTS ix_chunk_content_vector_cosine
        ON {doc_table_name}
        USING ivfflat (chunk_content_vector vector_cosine_ops)
        WITH (lists = {int(lists)});
    """
else:
    # For the doc_sample table
    index_schema_doc = f"""
        CREATE INDEX IF NOT EXISTS ix_chunk_content_vector_cosine ON {doc_table_name} USING ivfflat  (chunk_content_vector vector_cosine_ops);
    """

##FTS on text_Sample
index_schema_FTS = "CREATE INDEX IF NOT EXISTS idx_text_sample_content_gin ON text_sample USING GIN (to_tsvector('english', content))"


# Create HSNW indexes for the text_sample table
cursor.execute(index_schema_text)
print('Index ivfflat text_sample ')

# Create HSNW indexes for the doc_sample table
cursor.execute(index_schema_doc)
print('Index ivfflat doc_sample ')

# Perform Full-Text Search (Keyword Search) using PostgreSQL's FTS
cursor.execute(index_schema_FTS)
print('Index FTS text_sample ')

connection.commit()
cursor.close()
print("close connection.")

## hnsw is failing with the postgres 13 or 15 version with the vector extension enable. 
##UndefinedObject: access method "hnsw" does not exist
##The other option of index -  ivfflat  works but it seems it would not be suitable unless we have more data according tas per reference: 
# "a good place to start is rows / 1000 for up to 1M rows and sqrt(rows) for over 1M rows" (ref https://github.com/pgvector/pgvector) 
